# Show and Tell: A Neural Image Caption Generator

조용래, 김범중, 김지중 세 명은 지난 2015년에 발표된 구글의 논문 Show and Tell: A Neural Image Caption Generator를 읽고 이를 구현하기로 했습니다. 먼저 첫째 주인 오늘은 해당 논문을 리뷰하는 시간을 가져보기로 했습니다.

---

### Abstract

**What is "Show and Tell"?**

Show and Tell 논문은 이미지의 내용을 자동으로 설명해주는 모델을 제안합니다. Computer Vision과 NLP 기술을 사용하구요, 해당 영역에서의 급속한 발전에 따라 구현이 가능하게 된 거죠.

**How does it work?**

차차 설명하겠지만, 일단 여기서는 이미지에 대한 설명 sentence의 likelihood를 극대화하는 방식으로 학습된다고 정리하고 넘어가겠습니다.

**자랑질**

"우리 모델은 질적으로 양적으로 **ANOTHER LEVEL** 이다", "다양한 데이터 셋에 다양한 지표를 적용했더니 결과가 겁나 잘 나왔다"고 초록에서부터 자랑질을 시작합니다.

---

### 1. Introduction

Image captioning은 image classification이나 object detection처럼 컴퓨터 비전의 영역에서 주로 하는 task들에 비해 훨씬 어렵습니다. NLP까지 적용되니 당연히 그러겠죠. 이 모델은 일단 (1) 이미지에서 object를 잘 탐지해야 하구요, (2) 생성되는 문장이 내용적으로(semantically) 이미지를 충분히 설명해야 하고 (3) 문법적으로도 하자가 없어야 합니다. 기존의 모델 같은 경우에는 이런 각각의 sub-problem들의 해법 모델들을 갖다 붙인 거라면(stitch together), show and tell은 결합된 형태의 하나의 모델(a single joint model)입니다.이미지 정보 I를 input으로 받고 문장(단어의 시퀀스) S를 output으로 갖는데요,학습 방식은 아래와 같습니다.

$$\max p(S|I) \ s.t. \  S = \{S_1,S_2, ...\}$$

이 모델은 encoder-decoder 형태의 기계 번역 모델에서 착안된겁니다. 기계 번역의 경우, 아래와 같은 식을 목적으로 학습되는데요, T는 번역 되는 문장이고, S는 번역을 시키고자 하는 문장입니다.

$$\max p(T|S)$$

초기의 기계번역에서는 subtask를 쪼개고 따로 학습시키는 형태로 발전되어왔는데요, (당시 기준으로) 최근 "encoder"라는 RNN 모델이 S를 *읽고*, "decoder"라는 RNN 모델이 T를 *생성하는* 형태의 encoder-decoder가 훨씬 더 간단하게 모델로, 좋은 성과를 보일 수 있음을 증명했습니다.

간단하게 표현하자면 Show and Tell 모델은 여기서 input을 이미지로, encoder를 CNN으로 바꾼 형태입니다. 이를 구글에서는 NIC(Neural Image Caption)라고 명명했어요.

2절은 선행 연구인데요, 과감하게 제끼겠습니다.

---

### 3. Model

본격적으로 모델링에 대해서 알아봅시다.

**목적함수**
$$\theta^* = \arg\max_\theta \sum_{(I,S)} \log p(S|I;\theta)$$

$$\theta: parameter, I: Image, S: sentence $$


목적함수는 간단합니다. 모델이 계속 이미지를 읽어들이고 문장을 뱉어낼 텐데, 이 때 뱉어낸 문장이 정답 문장일 확률값을 최대화하도록 모델의 parameter를 update 해주는 겁니다. 여기서 S는 문장을 뜻하는데, 문장의 길이에는 당연히 제한이 없습니다. 문장 길이를 N이라고 해봅시다(N개의 단어로 구성된 문장).

$$S = \{S_0,S_1,S_2, ..., S_N\}$$

그럼, 이미지를 읽고 뱉어낸 문장이 정답 문장일 확률은 아래와 같이 다시 표현할 수 있습니다.

$$\log p(S|I) = \sum_{t=0}^N \log p(S_t|I,S_0,S_1,...,S_{t-1})$$

우변 로그 안쪽의 확률부터 살펴봅시다. t번째 단어로 정답 단어를 뱉을 확률은 애초에 읽어들였던 이미지 정보와 그 전 0,1,2,...,t-1번째 단어에 대한 조건부 확률로 표현이 되어있습니다. 참고로 0번째 단어는 문장 시작을 알리는 토큰이고, 여기에 대해서는 잠시 후에 다시 살펴볼거에요. 무튼 로그를 떼고 살펴보면 아래와 같은 식이 나오는데요.

$$p(S|I) = \prod_{t=0}^N p(S_t|I,S_0,S_1,...,S_{t-1})$$

문장의 각 단어가 발생할 확률이 서로 독립이라고 가정하고 문장의 확률값을 각 단어의 확률값의 곱으로 표현한 거네요.

**LSTM & CNN**
위와 같은 목적함수를 대입하기 적절한 모델은 RNN입니다. RNN은 sequence 형태의 데이터를 다루는 Neural Net입니다. hidden state를 갖는다는 특징이 있는데요, 이 hidden state는 일종의 "기억"입니다. 과거 데이터들이 집약되어있는 정보이지요. 새로운 정보를 받으면 과거 자신의 기억과 혼합하여 자신의 기억 상태를 갱신합니다.
<img src="https://scontent-hkg3-2.xx.fbcdn.net/v/t1.0-9/20597203_1549562975108540_8500664613978020758_n.jpg?oh=144bb21fc94ffbb8d0e0699ecd6f51d0&oe=5A36B873"></img>


<img src="https://scontent-hkg3-2.xx.fbcdn.net/v/t1.0-9/20621024_1549562955108542_4228195626921617948_n.jpg?oh=e5a480455d9082a37ca3cdf3198ff33b&oe=59EF0592"></img>
출처: [순환신경망(Recurrent neural networks) 개요  ](https://www.slideshare.net/ByoungHeeKim1/recurrent-neural-networks-73629152)

어쨌든간에 RNN은 새로운 input을 받으면 이 hidden state와 새로운 input 정보를 이용하여 무언가를 예측합니다. 위 슬라이드에서 확인하는 것 처럼 말입니다. NIC 모델에서는 다음 순서의 단어를 예측하겠죠. RNN을 좀 더 견고하게 굴리기 위해서, 구글은 decoder 모델로 LSTM을 선택했습니다. LSTM에 대해서는 다음 절에서 깊게 설명하겠습니다.

Encoder 모델은 당연히 이미지 패턴 인식 분야에서 쓰이는 CNN이 적용되었구요, 구체적으로는 당시 나온 지 얼마 안 되었던 batch normalization 레이어를 적용한, 그리고 2014년 이미지넷 챌린지에서 가장 좋은 성적을 거둔 모델을 썼다고 합니다.(2014년 우승 모델의 이름은 GoogLeNet입니다. 논문 정말 재수없게 쓰네요.)

#### 3.1. LSTM-based Sentence Generator

우선 LSTM의 탄생 배경에 대해 알아봅시다. 일반적인 RNN모형은 본질적으로 장기기억에 취약합니다. NLP에 적용하자면, So ... that ... 처럼 문장 안에서 멀리 떨어져서 호응하는 단어 관계에 대해서 파악할 수가 없다는 것입니다. 이는 backpropagation 과정에서 vanishing gradient 문제와 관련이 있습니다.

<img src="https://scontent-hkg3-2.xx.fbcdn.net/v/t1.0-9/20597074_1549562978441873_546351320538922452_n.jpg?oh=395df0d64a7db7bb7bc9e6409285c4d2&oe=59FC4340"></img>
출처: [RNN Tutorial Part 3 - BPTT와 Vanishing Gradient 문제](http://www.wildml.com/wp-content/uploads/2015/10/rnn-bptt-with-gradients.png)

위에서 확인할 수 있듯이, RNN의 backpropagation 과정에서는, chain rule에 의하여 직전 / 직후 hidden state끼리의 미분값이 계속 곱해집니다. 그렇기 때문에 학습이 진행되면 진행될 수록 sequence의 앞쪽 부분 가중치는 update 되지 않고, 결국 어떠한 예측값을 내놓을 때 앞쪽 부분의 데이터는 이용할 수가 없는 거죠.

LSTM은 이러한 문제를 해결하기 위해 고안되었습니다. 먼저, LSTM에서는 "memory cell"이라는 개념이 아까 hidden state라 불렸던 "기억 상태", 혹은 "지금껏 input들을 관찰한 결과 생성된 knowledge"의 역할을 합니다. 이러한 memory cell의 행동은 "gate"라는 layer에 의해서 컨트롤됩니다.

단순화 하자면 gate는 0 혹은 1로 구성된 벡터입니다. LSTM은 memory cell에 이러한 gate를 덧대어(elementwise-product를 적용하여) 과거 기억에서 어느 부분을 잊어야 할 지를 결정합니다. 이러한 gate를 forget 게이트라고 합니다. LSTM에는 forget 게이트를 포함하여 총 3개의 게이트가 있습니다.

* forget gate : 과거 memory cell의 value 중에서 무엇을 잊어야 할지 결정
* input gate : input value 중에서 어떤 것을 읽을 지 결정
* output gate : 현재 memory cell의 value 중에서 어떤 것을 출력시킬 지 결정.

수식을 통해 살펴보면 아래와 같습니다
<br>

$$ f_t = \sigma(W_{fx}x_t + W_{fm}m_{t-1})$$
$$ i_t = \sigma(W_{ix}x_t + W_{im}m_{t-1})$$
$$ o_t = \sigma(W_{ox}x_t + W_{om}m_{t-1})$$
$$ c_t = f_t \odot c_{t-1} + i_t \odot h(W_{cx}x_t + W_{cm}m_{t-1})$$
$$ m_t = o_t \odot c_t$$
$$ p_{t+1} = Softmax(m_t)$$
<br>
W들은 각각의 parameter matrix를 의미합니다. x는 input 벡터, m은 output 벡터입니다. 두 벡터들을 이용하여 forget, input, output 게이트가 구성됩니다. sigma 자리에는 sigmoid 함수가 적용됩니다. 즉, gate 벡터의 value들은 0과 1 근처의 값이 되겠죠.

LSTM은... 갓용래님이 잘 설명해주실거라 믿습니다 ㅠㅠ

**Training**

NIC의 Training 과정은 아래의 그림으로 요약됩니다.

<img src="https://scontent-hkg3-2.xx.fbcdn.net/v/t1.0-9/20526261_1549616661769838_2839910003471644545_n.jpg?oh=b51cac6d3da62db7d7a58c4238e64ad8&oe=59F99CA4"></imgs>

(1) 이미지를 CNN에 집어넣습니다.
(2) 마지막 레이어를 FC로 두어 이미지 embedding vector를 생성합니다.
(3) 이 때 CNN은 pretrained model을 사용합니다.
(4) 이를 LSTM에 집어넣어봅니다.
(5) 문장의 시작을 알리는 토큰인, S0의 embedding vector를 LSTM에 집어넣어봅니다.
(6) 첫 단어에 대한 확률 벡터가 출력됩니다. 확률값이 가장 높았던 단어를 S1로 고릅니다.
(7) S1의 embedding vector를 LSTM에 집어넣어봅니다.

이 과정을, 문장의 마지막 단어인 N번째 단어를 예측할때까지 반복합니다. 식으로 다시 한 번 표현해보면 아래와 같습니다.

$$ x_{-1} = CNN(I)$$
$$ x_t = W_eS_t, \ \ \ t \sim\{0,1,...,N-1\}$$
$$ p_{t+1} = LSTM(x_t)$$

I는 이미지, x는 LSTM의 input vector, 즉, 위에서 embedding vector라고 표현한 것들입니다. W는 embedding matrix겠죠. 여기서 한 가지 드는 의문이 생깁니다. 그림 정보를 맨 처음에만 집어넣어줘도 그림을 잘 설명하는 문장을 만들 수 있을까요? 정답은 (네) 입니다. LSTM에 이미지 임베딩 벡터를 반복적으로 넣어주면 모델이 지나치게 오버피팅된다고 합니다.

**Loss**

$$ L(I,S) = -\sum_{t=1}^N \log p_t(S_t)$$

negative log likelihood의 합을 loss function으로 둡니다. 이를 극소화하는 과정에서 LSTM의 모든 parameter, CNN의 마지막 레이어, 그리고 모든 워드임베딩 벡터가 학습됩니다.
<br>

**Inference**

학습된 모델을 통해 실제 예측을 진행하는 방법에는 두 가지가 있습니다.

(1) Sampling
한 단어씩 예측하는 방법입니다. LSTM에 넣으면 확률벡터가 나오는데, 이 확률벡터에서 가장 큰 확률값을 지니는 element를 선택하는 방법입니다.

(2) Beam Search
바둑을 둘 때 **몇 수 앞을 보고** 돌을 내려놓는 것과 같은 원리입니다. 제가 몇 글자 적는거 보다 아래 링크를 시청하시는 게 훨씬 이해가 빠를 것 같습니다.
[Udacity - Beam Search](https://www.youtube.com/watch?v=UXW6Cs82UKo)

---

### 4. Evaluation

#### 4.1 Evaluation Metrics
이런 저런 matrics를 통해 평가를 진행해봤다고 합니다.

#### 4.2 Datasets
<img src="https://scontent-hkg3-2.xx.fbcdn.net/v/t1.0-9/20604576_1549645541766950_7956015236653280323_n.jpg?oh=5c43ecc00317e051190c076f27753769&oe=59F432A3"></img>
데이터 셋 종류에는 이런 것들이 있다고 합니다. MSCOCO는 마소에서 관리하고 있는 데이터셋인데, 그만큼 훨씬 더 정제된 데이터라고 합니다.

#### 4.3 Results
데이터 셋의 크기 등 여러 요인에 변주를 주며 학습을 시켜보고, 결과를 비교해봤다고 합니다.

##### 4.3.1 Training Details
**오버피팅**

가장 먼저 부딪힌 문제는 오버피팅이었다고 합니다. supervised learning을 위해선 엄청나게 많은 사이즈의 데이터가 필요한데, 주어진 데이터셋 중에서 HQ의 데이터는 10만개 정도밖에 안된다고 합니다.

해결책 1) CNN은 pretrain 된 모델로 쓰자
해결책 2) drop out과 ensemble을 적용하자

두 방법 모두 효과가 있었다고 합니다.

**Other Details**

* optimizer : SGD
* learning rate : fixed
* embedding : 512-dimensions
* LSTM memory : 512-dimensions
* weight-init : randomly (CNN 제외)

#### 4.3.2 Generation Results
다른 모델들에 비해서 BLEU-1 score가 한참 높지만, 인간에 비해선 낮은 점수를 기록했다고 합니다.

#### 4.3.3 Transfer Learning Data Size and Label Quality
특정 데이터 셋에서 학습시킨 모델을 다른 데이터셋에 적용시켰을 때 성능이 얼마나 나올 지 한 번 시도해 봤다고 합니다. 이러한 실험에서 가장 좋은 성능을 보인 데이터 셋 변환은, Flickr30k랑 Flickr8k랑 바꿨을 때라고 합니다. label 방식이 유사할텐데, 당연한 거 아닙니까?? 어쨌든 이 경우에는 뭐 두 데이터셋을 혼용해서 쓰면 training량이 늘어나는거니깐 성능이 좋아진다고 합니다. MSCOCO랑 바꿔써봤더니, 단어 미스매치가 많았다고 합니다. 대체로 데이터 셋을 바꿔서 적용해보니 성능이 떨어졌습니다.


#### 4.3.4 Generation Diversity Discussion
NIC가 생성한 문장들은 질적으로 좀 괜찮을까요? 그리고 NIC는 다양한 문장들을 생성할 수 있을까요? 아래는 NIC의 N-best list입니다.
<img src="https://scontent-hkg3-2.xx.fbcdn.net/v/t1.0-9/20638237_1549657288432442_2886252960788682123_n.jpg?oh=2b7082bf1b5fc3ce5368312c3a97f814&oe=5A2B9991">
볼드체로 된 게 트레이닝셋에 없는 문장들이라고 합니다. 같은 이미지라 할지라도 다양한 측면에서 설명하고 있음을 확인할 수 있어요. 제일 잘 나온거 15문장을 뽑아서 BLEU score를 계산했더니 인간과 비슷했다고 합니다.



#### 4.3.7 Analysis of Embeddings
각 단어별 embedding vector에 대해서도 한 번 살펴보았더니, 잘 학습되었다고 합니다. Horse, Donkey, Pony 세 단어의 embedding vector가 유사함을 확인하였고, 이렇게 학습된 embedding vector는 CNN으로 하여금 더 말처럼 보이는 feature를 잘 찾도록 했다고 합니다.